Import all libraries needed for this project

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import difflib as dl
import requests 
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Scraping all data from the IGDB API.

In [2]:
# Your Client ID and Access Token (replace with actual values)
client_id = '2s6fe3ts3br2aikeue6v2mvpspjkf3'
access_token = 'grq8ke6nrir3w76qhto0f50zqoa2ql'

# IGDB API endpoint
url = 'https://api.igdb.com/v4/games'

# Headers for the request
headers = {
    'Client-ID': client_id,
    'Authorization': f'Bearer {access_token}',
}

# Fields you want to fetch for each game
fields = '''
fields id, name, rating, rating_count, age_ratings, aggregated_rating, aggregated_rating_count, 
alternative_names, artworks, bundles, category, checksum, collection, collections, cover, created_at, 
dlcs, expanded_games, expansions, external_games, first_release_date, follows, forks, franchise, franchises, 
game_engines, game_localizations, game_modes, genres, hypes, involved_companies, keywords, language_supports, 
multiplayer_modes, parent_game, platforms, player_perspectives, ports, release_dates, remakes, remasters, screenshots, 
slug, standalone_expansions, status, summary, tags, themes, total_rating, total_rating_count, updated_at, url, version_parent, version_title, videos, websites;
where rating != null & rating_count != null & total_rating != null & total_rating_count != null;
'''

In [3]:
def fetch_data(endpoint):
    url = f'https://api.igdb.com/v4/{endpoint}'
    response = requests.post(url, headers=headers, data='fields id, name; limit 500;')
    data = response.json()
    return {item['id']: item['name'] for item in data if 'id' in item and 'name' in item}

# Fetch relevant data for each field and create a mapping dictionary
franchise_dict = fetch_data('franchises')
game_engine_dict = fetch_data('game_engines')
cover_dict = fetch_data('covers')
game_localizations_dict = fetch_data('game_localizations')
game_modes_dict = fetch_data('game_modes')
companies_dict = fetch_data('companies')  # Involved companies data is in the 'companies' path
language_support_dict = fetch_data('language_supports')
multiplayer_modes_dict = fetch_data('multiplayer_modes')
platform_dict = fetch_data('platforms')
player_perspectives_dict = fetch_data('player_perspectives')
theme_dict = fetch_data('themes')

# Helper function to handle lists of IDs and convert them to names
def format_name_id(data_list, mapping_dict=None):
    if isinstance(data_list, list):  # If it's a list
        formatted_items = []
        for item in data_list:
            if isinstance(item, int):  # Handle case where it's just an ID
                formatted_items.append(f"{mapping_dict.get(item, 'Unknown')} ({item})" if mapping_dict else str(item))
        return '|'.join(formatted_items)
    elif isinstance(data_list, int):  # If it's just a single ID
        return f"{mapping_dict.get(data_list, 'Unknown')} ({data_list})" if mapping_dict else str(data_list)
    return 'N/A'

# Function to fetch all games with pagination and store each game's details in a list
def fetch_all_games(limit=500, fields=''):
    all_games = []
    offset = 0
    game_count = 0  # To keep track of the total number of games

    while True:
        # Build the body with limit and offset to paginate through results
        body = f'{fields} limit {limit}; offset {offset};'
        
        # Make the API request
        response = requests.post('https://api.igdb.com/v4/games', headers=headers, data=body)
        
        # Check if the request was successful
        if response.status_code == 200:
            games = response.json()
            
            # Break the loop if no more games are returned
            if not games:
                print("No more games to fetch.")
                break
            
            # Process each game's details and store them in a list of formatted strings
            for game in games:
                game_info = {
                    'ID': str(game.get('id', 'N/A')),
                    'Name': str(game.get('name', 'N/A')),
                    'Rating': str(game.get('rating', 'N/A')),
                    'Rating Count': str(game.get('rating_count', 'N/A')),
                    'Age Ratings': format_name_id(game.get('age_ratings', [])),
                    'Aggregated Rating': str(game.get('aggregated_rating', 'N/A')),
                    'Aggregated Rating Count': str(game.get('aggregated_rating_count', 'N/A')),
                    'Alternative Names': format_name_id(game.get('alternative_names', [])),
                    'Artworks': format_name_id(game.get('artworks', [])),
                    'Bundles': format_name_id(game.get('bundles', [])),
                    'Category': str(game.get('category', 'N/A')),
                    'Checksum': str(game.get('checksum', 'N/A')),
                    'Collection': str(game.get('collection', 'N/A')),
                    'Collections': format_name_id(game.get('collections', [])),
                    'Cover': str(game.get('cover', 'N/A')),
                    'Cover Name': cover_dict.get(game.get('cover'), 'Unknown'),
                    'Created At': str(game.get('created_at', 'N/A')),
                    'DLCs': format_name_id(game.get('dlcs', [])),
                    'Expanded Games': format_name_id(game.get('expanded_games', [])),
                    'Expansions': format_name_id(game.get('expansions', [])),
                    'External Games': format_name_id(game.get('external_games', [])),
                    'First Release Date': str(game.get('first_release_date', 'N/A')),
                    'Follows': str(game.get('follows', 'N/A')),
                    'Forks': format_name_id(game.get('forks', [])),
                    'Franchise': format_name_id(game.get('franchise', [])),
                    'Franchise Name': format_name_id(game.get('franchise', []), mapping_dict=franchise_dict),
                    'Game Engines': format_name_id(game.get('game_engines', [])),
                    'Game Engine Name': format_name_id(game.get('game_engines', []), mapping_dict=game_engine_dict),
                    'Game Localizations': format_name_id(game.get('game_localizations', [])),
                    'Game Modes': format_name_id(game.get('game_modes', []), mapping_dict=game_modes_dict),
                    'Genres': format_name_id(game.get('genres', [])),
                    'Hypes': str(game.get('hypes', 'N/A')),
                    'Involved Companies': format_name_id(game.get('involved_companies', [])),
                    'Company Names': format_name_id(game.get('involved_companies', []), mapping_dict=companies_dict),
                    'Keywords': format_name_id(game.get('keywords', [])),
                    'Language Supports': format_name_id(game.get('language_supports', []), mapping_dict=language_support_dict),
                    'Multiplayer Modes': format_name_id(game.get('multiplayer_modes', []), mapping_dict=multiplayer_modes_dict),
                    'Platforms': format_name_id(game.get('platforms', []), mapping_dict=platform_dict),
                    'Player Perspectives': format_name_id(game.get('player_perspectives', []), mapping_dict=player_perspectives_dict),
                    'Ports': format_name_id(game.get('ports', [])),
                    'Release Dates': format_name_id(game.get('release_dates', [])),
                    'Remakes': format_name_id(game.get('remakes', [])),
                    'Remasters': format_name_id(game.get('remasters', [])),
                    'Screenshots': format_name_id(game.get('screenshots', [])),
                    'Slug': str(game.get('slug', 'N/A')),
                    'Standalone Expansions': format_name_id(game.get('standalone_expansions', [])),
                    'Status': str(game.get('status', 'N/A')),
                    'Storyline': str(game.get('storyline', 'N/A')),
                    'Tags': format_name_id(game.get('tags', [])),
                    'Themes': format_name_id(game.get('themes', []), mapping_dict=theme_dict),
                    'Theme Name': format_name_id(game.get('themes', []), mapping_dict=theme_dict),
                    'Total Rating': str(game.get('total_rating', 'N/A')),
                    'Total Rating Count': str(game.get('total_rating_count', 'N/A')),
                    'Updated At': str(game.get('updated_at', 'N/A')),
                    'URL': str(game.get('url', 'N/A')),
                    'Version Parent': format_name_id(game.get('version_parent', [])),
                    'Version Title': str(game.get('version_title', 'N/A')),
                    'Videos': format_name_id(game.get('videos', [])),
                    'Websites': format_name_id(game.get('websites', []))
                }
                all_games.append(game_info)
            
            # Update the total game count
            game_count += len(games)
            
            # Increment the offset for the next batch of games
            offset += limit
            
            # Pause to respect rate limits
            time.sleep(0.3)  # Wait 300ms between requests (4 requests per second limit)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            print(response.text)
            break

    print(f"Total games fetched: {game_count}")
    return all_games

# Fetch all the games
games_data = fetch_all_games(limit=500, fields=fields)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(games_data)

# Save the DataFrame to a CSV file
df.to_csv('igdb_games_data.csv', index=False)

print(f"Game data saved to igdb_games_data.csv with {len(df)} games.")

No more games to fetch.
Total games fetched: 30873
Game data saved to igdb_games_data.csv with 30873 games.


Lets Visualize the table and see how it turned out so far

In [ ]:
games_dat = pd.read_csv('igdb_games_data.csv')
games_dat.head()

Lets clean up duplicate values in the coloumns we specially got ID's and names. 

In [4]:
# Function to remove duplicates and keep only one column
def clean_duplicate_columns(df, id_column, name_column):
    """
    Remove one of the columns (ID or Name) to avoid duplication.
    If both columns exist, keep only the Name column.
    """
    # Check if both columns exist
    if id_column in df.columns and name_column in df.columns:
        # Drop the ID column, keeping only the Name column
        df.drop(columns=[id_column], inplace=True)

# List of column pairs to clean (ID, Name)
columns_to_clean = [
    ('Themes', 'Theme Name'),
    ('Platforms', 'Platform Name'),
    ('Franchise', 'Franchise Name'),
    ('Game Engines', 'Game Engine Name'),
    ('Involved Companies', 'Company Names'),
    ('Genres', 'Genre Name'),
    # Add other ID-Name column pairs here
]

# Apply the function to clean the duplicate columns for all relevant pairs
for id_col, name_col in columns_to_clean:
    clean_duplicate_columns(df, id_col, name_col)

# Save the cleaned DataFrame to a CSV file
df.to_csv('igdb_games_data.csv', index=False)

print("Cleaned file saved successfully.")

Cleaned file saved successfully.


Lets Visualize the table again

In [5]:
games_dat.head()

,ID,Name,Rating,Rating Count,Age Ratings,Aggregated Rating,Aggregated Rating Count,Alternative Names,Artworks,Bundles,...,Tags,Theme Name,Total Rating,Total Rating Count,Updated At,URL,Version Parent,Version Title,Videos,Websites
0,215217,Tiny Rogues,91.641903,5,162917,NaN,NaN,NaN,86350,NaN,...,1|17|268435461|268435468|268435487|268435488|2...,Action (1)|Fantasy (17),91.641903,5,1725962367,https://www.igdb.com/games/tiny-rogues,NaN,NaN,74227|104239,307462|307463|501746|605460
1,82461,10tons Adventure Puzzles Bundle,60.000000,0,84506,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,60.000000,0,1670990469,https://www.igdb.com/games/10tons-adventure-pu...,NaN,NaN,NaN,440080
2,236547,Thronefall,77.856060,14,NaN,NaN,NaN,NaN,NaN,NaN,...,1|17|21|268435471|268435480|268435488,Action (1)|Fantasy (17)|Survival (21),77.856060,14,1725962530,https://www.igdb.com/games/thronefall,NaN,NaN,97151|118938,528578|556705|570629
3,118128,Little Dungeon Stories,50.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,...,268435468|268435471|268435487|268435488,NaN,50.000000,0,1713262585,https://www.igdb.com/games/little-dungeon-stories,NaN,NaN,NaN,109257|391447
4,10402,Heavy Bullets,60.467833,7,NaN,80.0,1.0,NaN,9037,NaN,...,1|18|27|268435461|268435488|536871038|53687132...,Action (1)|Science fiction (18)|Comedy (27),70.233917,8,1719999714,https://www.igdb.com/games/heavy-bullets,NaN,NaN,5140,44116|44117|238156|397643


Some 